[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PLN-disca-iimas/DigitalHumanitiesSchool/blob/main/demo-transformers_classification.ipynb)

# Clasificación de textos con 🤗 Transformers

> Cómo afinar un modelo de distilbert para clasificar los tweets del TASS.

## Configuración

Si está ejecutando este notebook en Google Colab, ejecute la siguiente celda para instalar las bibliotecas que necesitamos:

In [ ]:
!pip install evaluate
!pip install transformers==4.28.0
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.8 MB/s eta 0:00:00


Para compartir tu modelo con la comunidad, primero crea una cuenta en el [Hugging Face Hub](https://huggingface.co/join). A continuación, ejecute la siguiente celda y proporcione su nombre de usuario y contraseña para generar un token de autenticación:

In [ ]:
# Esto sólo funciona en Google Colab! Para los notebooks normales, es necesario ejecutar esto en el terminal
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

Si no tienes instalado [Git LFS](https://git-lfs.github.com), puedes hacerlo descomentando y ejecutando la celda de abajo:

In [ ]:
!apt install git-lfs
!git config --global user.email "helena.adorno@gmail.com"
!git config --global user.name "helenpy"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
#si usamos colab
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 10.0 MB/s eta 0:00:00


## Cargar y explorar los datos

Utilizaremos 🤗 Datasets para cargar y procesar nuestro conjunto de datos.

In [ ]:
from datasets import load_dataset, Dataset, Features, ClassLabel, Sequence, Value, DatasetDict
import pandas as pd

# Lectura del archivo donde se encuentran los datos de entrenamiento y validación
data = pd.read_csv('/content/gdrive/MyDrive/Meia2023/Modulo2-ClasificacionTextos/corpusTASS-2020/train.tsv', sep='\t')
data_dev = pd.read_csv('/content/gdrive/MyDrive/Meia2023/Modulo2-ClasificacionTextos/corpusTASS-2020/dev.tsv', sep='\t')

data = data.drop(columns=['id', 'pais'])
data_dev = data_dev.drop(columns=['id', 'pais'])

# Diccionario de mapeo de etiquetas
mapeo_etiquetas = {'N': 0, 'NEU': 1, 'P': 2}

# Transformación de la columna "etiqueta"
#data['etiqueta_num'] = data['etiqueta'].map(mapeo_etiquetas)
#data_dev['etiqueta_num'] = data_dev['etiqueta'].map(mapeo_etiquetas)

#definimos la estructura del dataset de Huggingface
features = Features({
    'texto': Value('string'),
    'etiqueta': ClassLabel(num_classes=3, names=['P', 'N', 'NEU'])
})

#transformamos de dataframe a dataset
dataset_train = Dataset.from_pandas(data, features=features)
dataset_dev = Dataset.from_pandas(data_dev, features=features)

dataset_train = dataset_train.rename_column("texto", "text")
dataset_train = dataset_train.rename_column("etiqueta", "label")
dataset_dev = dataset_dev.rename_column("texto", "text")
dataset_dev = dataset_dev.rename_column("etiqueta", "label")

#creamos diccionario de dataset
dataset = DatasetDict({'train': dataset_train, 'test': dataset_dev})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4802
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2443
    })
})

In [ ]:
import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "Taken from https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset['train'])

,text,label
0,@adri_buzo @FulgenLison Nop. Pero ahora me queda más claro el asunto Gracias !,P
1,Trabajando con el equipo de @Juliaenlaonda para el especial del próximo jueves. Ya os adelanto que va a ser un programón!,P
2,Que horrible ser tan Celosa,N
3,esta estupida de Abis a la par mía esperando a que saquen el desgraciado Trailer de Resident Evil fuck hasta as 12 md LA vamoooos,P
4,"Piensa en grande y tus hechos crecerán, Piensa que puedes y podrás. Todo esta en el estado mental. #BuenFinDeSemana",P
5,Por qué un partido a las 10 a.m.,NEU
6,"@cuervotinelli @demianchildemb1 Un solo corazón !!. Te amo Viejo :*, que descanses, me fui a dormir... Me gustaría soñar con vos",P
7,"Todo el mundo me dice que soy re enana en los bailes, yo no entiendo, si estoy yendo con championes de plataforma",N
8,@hackandbeers Muy buenas amigos! Como podemos contactar con ustedes?,NEU
9,"No me gusta el maíz e iba a cenar mazorca, me dice mi madre que se ha puesto mala y que no me la coma. Y ME HE TENIDO QUE HACER UNA PIZZA",N


In [ ]:
dataset.set_format("pandas")
df = dataset['train'][:]
df.head()

,text,label
0,@morbosaborealis jajajaja... eso es verdad... ...,1
1,@Adriansoler espero y deseo que el interior te...,2
2,"comprendo que te molen mis tattoos, pero no te...",2
3,"Mi última partida jugada, con Sona support. La...",0
4,Tranquilos que con el.dinero de Camacho seguro...,0


In [ ]:
dataset.reset_format()

In [ ]:
show_random_elements(dataset['train'], num_examples=3)

,text,label
0,"@ROSYARANGO dicen que cuidar a un recién nacido es trabajo en equipo, así que yo le cambio el pañal",P
1,"Tengo mi lado lindo y mi lado culero, pero bien culero.",N
2,No es posible que justo hoy me sienta enferma.,N


## Tokenizar las reseñas

In [ ]:
from transformers import AutoTokenizer

#model_checkpoint = "vg055/roberta-base-bne-finetuned-Tass2020"
model_checkpoint = "distilbert-base-multilingual-cased"#distilbert-base-multilingual-cased #vg055/roberta-base-bne-finetuned-Tass2020
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Tokenización de las reseñas

In [ ]:
tokenizer.vocab_size

119547

In [ ]:
text = "¡hola, estamos muy felices practicando la tokenizacion!"
tokenized_text = tokenizer.encode(text)

for token in tokenized_text:
    print(token, tokenizer.decode([token]))

101 [CLS]
199 ¡
110516 hol
10113 ##a
117 ,
11504 esta
13386 ##mos
13436 muy
13077 fel
39801 ##ices
56309 prac
13640 ##tica
10605 ##ndo
10109 la
18436 tok
18687 ##eni
104679 ##zaci
10263 ##on
106 !
102 [SEP]


In [ ]:
encoded_text = tokenizer(text, return_tensors="pt")
encoded_text

{'input_ids': tensor([[   101,    199, 110516,  10113,    117,  11504,  13386,  13436,  13077,
          39801,  56309,  13640,  10605,  10109,  18436,  18687, 104679,  10263,
            106,    102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
def tokenize_reviews(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
columns = dataset['train'].column_names
columns.remove("label")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

Map:   0%|          | 0/4802 [00:00<?, ? examples/s]

Map:   0%|          | 0/2443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 4802
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2443
    })
})

In [ ]:
encoded_dataset['train'][0]

{'label': 1,
 'input_ids': [101,
  137,
  24984,
  19804,
  107956,
  33269,
  10201,
  10320,
  10320,
  10320,
  119,
  119,
  119,
  36584,
  10196,
  79381,
  119,
  119,
  119,
  36579,
  10192,
  13605,
  11381,
  10854,
  13819,
  10133,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

## Cargar el modelo preentrenado

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.we

### De las input IDs a los hidden states

In [ ]:
outputs = model(**encoded_text)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0558,  0.1671, -0.1654]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Definir las métricas de rendimiento

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import transformers
transformers.__version__

'4.28.0'

## Afinar el modelo preentrenado

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 12
num_train_epochs=3
train_dataset = encoded_dataset["train"]
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results-meia_2",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=False#,
    #push_to_hub_model_id=f"{model_name}-finetuned-tass"
)

In [ ]:
from transformers import Trainer

test_dataset = encoded_dataset["test"]

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.935000,0.928102,0.556283
2,0.755500,0.918149,0.587802
3,0.625500,0.954801,0.592305


TrainOutput(global_step=1203, training_loss=0.8002919920860284, metrics={'train_runtime': 167.9182, 'train_samples_per_second': 85.792, 'train_steps_per_second': 7.164, 'total_flos': 168645907363644.0, 'train_loss': 0.8002919920860284, 'epoch': 3.0})

## Guarda modelo (local) y realiza predicciones

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/")

In [ ]:
from transformers import AutoModelForSequenceClassification
model2 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/")

In [ ]:
from transformers import pipeline
tx = pipeline("text-classification", model=model2, tokenizer=tokenizer)
texto = "Estoy muy feliz aprendiendo en el Escuela de Verano en Humanidades Digitales"
tx(texto)

[{'label': 'LABEL_0', 'score': 0.8999026417732239}]

## OPCIONAL: Empuje hacia el Hugging Face Hub

Si te conectaste a tu cuenta de Huggingface y en el trainer activaste los parámetros de push_to_hub y push_to_hub_model_id, entonces puedes subir tu modelo al repositorio de Huggingface, realizando lo siguiente

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file runs/Jun21_14-41-16_73a0e079d595/events.out.tfevents.1687358491.73a0e079d595.282.0:   0%|         …

To https://huggingface.co/helenpy/distilbert-base-uncased-finetuned-tass
   63f8492..f759890  main -> main

   63f8492..f759890  main -> main

To https://huggingface.co/helenpy/distilbert-base-uncased-finetuned-tass
   f759890..2dc7f05  main -> main

   f759890..2dc7f05  main -> main



'https://huggingface.co/helenpy/distilbert-base-uncased-finetuned-tass/commit/f75989096eb8fa1984e6d4c28a92e6127a81a908'

## Descargue el modelo desde el Hub

In [ ]:
from transformers import pipeline

model_checkpoint = "helenpy/distilbert-base-uncased-finetuned-tass"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

In [ ]:
pipe("estoy muy contento de estudiar en este curso")

[{'label': 'LABEL_1', 'score': 0.6821224093437195}]